In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Plan

I. Cleaning data. 

0. Rename columns and delete "?" and replace "spaces" to "_". Do it with Python for.
1. Find how much values each og five ranges contain. Ranges are 1st range: from min to min + 0.2*(max - min), 2nd range: from min + 0.2*(max - min) to min + 0.4*(max - min), 3rd range: from min + 0.4*(max - min) to min + 0.6*(max - min), 4th range: from min + 0.6*(max - min) to min + 0.8*(max - min), 5th range: from min + 0.8*(max - min) to max. 
2. Drop all columns which values group in one of range. For example, each column contains 6819 values. In column 'Operating_Profit_Rate' almost values (6817) group in 5th range as for "Bankrapcy == 0" as for "Bankrapcy == 1" Such columns contains an empty (usefulness) information for us. Therefore I drop them. 
3. Build sns.pairplot to visualize intermediate result.


II. Learn simple ML-model with GridSearchCV, RandomizedSearchCV, RandomForestClassifier. Build a Decision Tree.

Calculate Precision Score, Recall Score and F1 Score. Plot Confision Matrix. 

Plot a pie chart of features importances. 


# Conclusion. 


The most important feature is "Working Capital to Total Assets". It have got maximun weight about 26.7% to determine bankruptcy of company. 


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("../input/company-bankruptcy-prediction/data.csv")
df.head(10)

In [ ]:
df.describe()

In [ ]:
df[df.columns[1]].mean()

In [ ]:
# iterating the columns 
i = 0
for i in range(len(df.columns)): 
    print(df.columns[i]) 
    #print("Max - Min:", df[df.columns[i]].max() - df[df.columns[i]].min())
    #print("(Max - Min) / Mean:", (df[df.columns[i]].max() - df[df.columns[i]].min() / df[df.columns[i]].mean()))

Rename columns. Eliminate spaces. 

In [ ]:
i = 0
for i in range(len(df.columns)): 
    old_name = df.columns[i]
    new_name = df.columns[i].strip().replace(" ", "_")

    df.rename(columns={old_name: new_name}, inplace = True)

    print(df.columns[i], ":", "Mean:", df[df.columns[i]].mean())

In [ ]:
df.rename(columns={"Bankrupt?": "Bankrupt"}, inplace = True)

In [ ]:
df_bankrupt_false = df.loc[df.Bankrupt == 0]
df_bankrupt_true = df.loc[df.Bankrupt == 1]

In [ ]:
plt.figure(figsize=(15,8))
sns.scatterplot(data=df, x="ROA(C)_before_interest_and_depreciation_before_interest", 
                y="Cash_Flow_Per_Share", hue="Bankrupt")

In [ ]:
plt.figure(figsize=(15,8))
sns.histplot(data=df_bankrupt_false, x="ROA(C)_before_interest_and_depreciation_before_interest")

Build Seaborn ScatterPlot on data. We will take pair of columns. 
x = "ROA(C)_before_interest_and_depreciation_before_interest"
y - each another column

Build ScatterPlot with code

sns.scatterplot(data=df, 
                x="ROA(C)_before_interest_and_depreciation_before_interest", 
                y="Tax_rate_(A)", hue="Bankrupt")

get image

![https://ibb.co/cykbQMR](https://ibb.co/cykbQMR)

then

build ScatterPlot with code

sns.scatterplot(data=df, 
                x="ROA(C)_before_interest_and_depreciation_before_interest", 
                y="Operating_Profit_Rate", hue="Bankrupt")
                
![https://ibb.co/Tg42HhT](https://ibb.co/Tg42HhT)

At last image we can see practically straight line at the top. I don't think that such line will help predict bancrupcy. 

If you can not see images you can run code to get them below. 

I checked all pair of columns and dpor all columns which won't help us. 

In [ ]:
plt.figure(figsize=(15,8))
sns.scatterplot(data=df, 
                x="ROA(C)_before_interest_and_depreciation_before_interest", 
                y="Operating_Profit_Rate", hue="Bankrupt")

In [ ]:
plt.figure(figsize=(15,8))
sns.scatterplot(data=df, 
                x="ROA(C)_before_interest_and_depreciation_before_interest", 
                y="Tax_rate_(A)", hue="Bankrupt")

The most valueable pairs with "ROA(C)_before_interest_and_depreciation_before_interest" are
* Operating_Expense_Rate
* Research_and_development_expense_rate
* Interest-bearing_debt_interest_rate
* Tax_rate_(A)

I make 2 new DataFrames: most_df and wo_week_df.

*most_df* contains the most valueable columns.

*wo_week_df* does not contain the weekest columns such as 
*     Pre-tax_net_Interest_Rate
*     Operating_Profit_Rate
*     After-tax_net_Interest_Rate
*     Non-industry_income_and_expenditure/revenue
*     Continuous_interest_rate_(after_tax)
*     Revenue_Per_Share_(Yuan_¥)
*     Realized_Sales_Gross_Profit_Growth_Rate
    
And then I will work with these two DF. 

In [ ]:
plt.figure(figsize=(15,8))
sns.scatterplot(data=df, 
                x="ROA(C)_before_interest_and_depreciation_before_interest", 
                y="Quick_Assets/Current_Liability", hue="Bankrupt")

In [ ]:
plt.figure(figsize=(15,8))
sns.scatterplot(data=df, 
                x="ROA(C)_before_interest_and_depreciation_before_interest", 
                y="Cash/Current_Liability", hue="Bankrupt")

In [ ]:
# Each column contents values from min_value to max_value, for example, from 0 to 1, as for column "ROA(C)_before_interest_and_depreciation_before_interest"
# Distribute values for 5 groups:
# 1st group: all values from 0 to 0.2, 5th group: all values from 0.8 to 1
# It is not a percentiles. It's important. 

# Find border values for each group for data in column "ROA(C)_before_interest_and_depreciation_before_interest"

min_value = df['ROA(C)_before_interest_and_depreciation_before_interest'].min()
max_value = df['ROA(C)_before_interest_and_depreciation_before_interest'].max()

mm_range = max_value - min_value
mm_range

q00 = min_value
q20 = min_value + mm_range * 0.2
q40 = min_value + mm_range * 0.4
q60 = min_value + mm_range * 0.6
q80 = min_value + mm_range * 0.8
q100 = max_value

print(q00, q20, q40, q60, q80, q100)

In [ ]:
# Print count of values in each group

print(df['ROA(C)_before_interest_and_depreciation_before_interest'].count())
print()
print("0-.2", ".2-.4", ".4-.6", ".6-.8", " .8-1")
print(df.loc[(df['ROA(C)_before_interest_and_depreciation_before_interest'] >= q00) & (df['ROA(C)_before_interest_and_depreciation_before_interest'] <= q20)]['ROA(C)_before_interest_and_depreciation_before_interest'].count(), 
      df.loc[(df['ROA(C)_before_interest_and_depreciation_before_interest'] >= q20) & (df['ROA(C)_before_interest_and_depreciation_before_interest'] <= q40)]['ROA(C)_before_interest_and_depreciation_before_interest'].count(),
      df.loc[(df['ROA(C)_before_interest_and_depreciation_before_interest'] >= q40) & (df['ROA(C)_before_interest_and_depreciation_before_interest'] <= q60)]['ROA(C)_before_interest_and_depreciation_before_interest'].count(),
      df.loc[(df['ROA(C)_before_interest_and_depreciation_before_interest'] >= q60) & (df['ROA(C)_before_interest_and_depreciation_before_interest'] <= q80)]['ROA(C)_before_interest_and_depreciation_before_interest'].count(),
      df.loc[(df['ROA(C)_before_interest_and_depreciation_before_interest'] >= q80) & (df['ROA(C)_before_interest_and_depreciation_before_interest'] <= q100)]['ROA(C)_before_interest_and_depreciation_before_interest'].count(),
     sep="  ")

In [ ]:
# Find border values for each group for data in column "Operating_Expense_Rate"

min_value = df['Operating_Expense_Rate'].min()
max_value = df['Operating_Expense_Rate'].max()

mm_range = max_value - min_value
mm_range

q00 = min_value
q20 = min_value + mm_range * 0.2
q40 = min_value + mm_range * 0.4
q60 = min_value + mm_range * 0.6
q80 = min_value + mm_range * 0.8
q100 = max_value

print(q00, q20, q40, q60, q80, q100)

In [ ]:
# Print count of values in each group

print(df['Operating_Expense_Rate'].count())
print()
print("0-.2", ".2-.4", ".4-.6", ".6-.8", " .8-1")
print(df.loc[(df['Operating_Expense_Rate'] >= q00) & (df['Operating_Expense_Rate'] <= q20)]['Operating_Expense_Rate'].count(), 
      df.loc[(df['Operating_Expense_Rate'] >= q20) & (df['Operating_Expense_Rate'] <= q40)]['Operating_Expense_Rate'].count(),
      df.loc[(df['Operating_Expense_Rate'] >= q40) & (df['Operating_Expense_Rate'] <= q60)]['Operating_Expense_Rate'].count(),
      df.loc[(df['Operating_Expense_Rate'] >= q60) & (df['Operating_Expense_Rate'] <= q80)]['Operating_Expense_Rate'].count(),
      df.loc[(df['Operating_Expense_Rate'] >= q80) & (df['Operating_Expense_Rate'] <= q100)]['Operating_Expense_Rate'].count(),
     sep="  ")

# See output
# df['Operating_Expense_Rate'].count() = 6819
# Count of values in range from 0.0 to 0.2 = 4818 or 4818 / 6819 = 0.7, or 70%

In [ ]:
column_name = 'Current_Assets/Total_Assets'

# Find border values for each group for data in column "Operating_Expense_Rate"

min_value = df[column_name].min()
max_value = df[column_name].max()

mm_range = max_value - min_value
mm_range

q00 = min_value
q20 = min_value + mm_range * 0.2
q40 = min_value + mm_range * 0.4
q60 = min_value + mm_range * 0.6
q80 = min_value + mm_range * 0.8
q100 = max_value

print(q00, q20, q40, q60, q80, q100)

# Print count of values in each group

print(df[column_name].count())
print()
print("0-.2", ".2-.4", ".4-.6", ".6-.8", " .8-1")
print(df.loc[(df[column_name] >= q00) & (df[column_name] <= q20)][column_name].count(), 
      df.loc[(df[column_name] >= q20) & (df[column_name] <= q40)][column_name].count(),
      df.loc[(df[column_name] >= q40) & (df[column_name] <= q60)][column_name].count(),
      df.loc[(df[column_name] >= q60) & (df[column_name] <= q80)][column_name].count(),
      df.loc[(df[column_name] >= q80) & (df[column_name] <= q100)][column_name].count(),
     sep="  ")


In [ ]:
# Find border values for each group for data in column "Pre-tax_net_Interest_Rate"

min_value = df['Pre-tax_net_Interest_Rate'].min()
max_value = df['Pre-tax_net_Interest_Rate'].max()

mm_range = max_value - min_value
mm_range

q00 = min_value
q20 = min_value + mm_range * 0.2
q40 = min_value + mm_range * 0.4
q60 = min_value + mm_range * 0.6
q80 = min_value + mm_range * 0.8
q100 = max_value

print(q00, q20, q40, q60, q80, q100)

In [ ]:
# Print count of values in each group

print(df['Pre-tax_net_Interest_Rate'].count())

print()

print("0-.2", ".2-.4", ".4-.6", ".6-.8", "  .8-1")

print(df.loc[(df['Pre-tax_net_Interest_Rate'] >= q00) & (df['Pre-tax_net_Interest_Rate'] <= q20)]['Pre-tax_net_Interest_Rate'].count(), 
      df.loc[(df['Pre-tax_net_Interest_Rate'] >= q20) & (df['Pre-tax_net_Interest_Rate'] <= q40)]['Pre-tax_net_Interest_Rate'].count(),
      df.loc[(df['Pre-tax_net_Interest_Rate'] >= q40) & (df['Pre-tax_net_Interest_Rate'] <= q60)]['Pre-tax_net_Interest_Rate'].count(),
      df.loc[(df['Pre-tax_net_Interest_Rate'] >= q60) & (df['Pre-tax_net_Interest_Rate'] <= q80)]['Pre-tax_net_Interest_Rate'].count(),
      df.loc[(df['Pre-tax_net_Interest_Rate'] >= q80) & (df['Pre-tax_net_Interest_Rate'] <= q100)]['Pre-tax_net_Interest_Rate'].count(),
     sep="     ")

# See output
# df['Pre-tax_net_Interest_Rate'].count() = 6819
# Count of values in range from 0.6 to 0.8 = 6796 or 6796 / 6819 = 0.9966270714180965, more than 99%

In [ ]:
# Check our method for one more column "Operating_Profit_Rate"
# Find border values for each group for data in column "Operating_Profit_Rate"

min_value = df['Operating_Profit_Rate'].min()
max_value = df['Operating_Profit_Rate'].max()

mm_range = max_value - min_value
mm_range

q00 = min_value
q20 = min_value + mm_range * 0.2
q40 = min_value + mm_range * 0.4
q60 = min_value + mm_range * 0.6
q80 = min_value + mm_range * 0.8
q100 = max_value

print(q00, q20, q40, q60, q80, q100)

In [ ]:
# Print count of values in each group

print(df['Operating_Profit_Rate'].count())

print()

print("0-.2", ".2-.4", ".4-.6", ".6-.8", "  .8-1")

print(df.loc[(df['Operating_Profit_Rate'] >= q00) & (df['Operating_Profit_Rate'] <= q20)]['Operating_Profit_Rate'].count(), 
      df.loc[(df['Operating_Profit_Rate'] >= q20) & (df['Operating_Profit_Rate'] <= q40)]['Operating_Profit_Rate'].count(),
      df.loc[(df['Operating_Profit_Rate'] >= q40) & (df['Operating_Profit_Rate'] <= q60)]['Operating_Profit_Rate'].count(),
      df.loc[(df['Operating_Profit_Rate'] >= q60) & (df['Operating_Profit_Rate'] <= q80)]['Operating_Profit_Rate'].count(),
      df.loc[(df['Operating_Profit_Rate'] >= q80) & (df['Operating_Profit_Rate'] <= q100)]['Operating_Profit_Rate'].count(),
     sep="     ")

# See output
# df['Operating_Profit_Rate'].count() = 6819
# Count of values in range from 0.8 to 1.0 = 6817 or 6817 / 6819 = 0.9997067018624431, more than 99%

I guess if count of values in one of range {from 0.0 to 0.2, from 0.2 to 0.4, from 0.4 to 0.6, from 0.6 to 0.8, from 0.8 to 1.0} are more than 95% of total count of values in column than this column we can drop. This column will give us never usefull data. 


When data concentrate around one value it means that all of companies have a similaraty in factors. Therefore, we cann't predict bankrupcy using them. 


Create new DataFrame *df_drop* and drop all columns which have got concenration in values. 

For example, we have a column which contains values and count values:

Value              Count            % of total count (6819) of values in column

0.0 to 0.2          446                        6.54

0.2 to 0.4         1748                       25.63

0.4 to 0.6         2113                       30.98

0.6 to 0.8         1627                       23.86

0.8 to 1.0          885                       12.98


We don't drop such columns like column in an example. 

We drop columns which on of range has % of total count '>= 75%' (or '>= 0.75'). 


You can see code below. 

In [ ]:
df_drop = df
df_drop.head(10)

In [ ]:
flag = False

i = 0

total_columns = len(df.columns)
print(total_columns)
print()

count_total = 6819
# count_total = df_drop[df_drop.columns[i]].count()

for i in range(1, total_columns):
    for j in range(0, 10, 2):
        min_value = df[df.columns[i]].min() # We need to work with source DataFrame because 
        max_value = df[df.columns[i]].max() # if we work with df_drop each step which delete columns shift their numbers
        
        print("i:", i, "min:", min_value, "max:", max_value)
        
        mm_range = max_value - min_value
        
        qlow = min_value + mm_range * (j / 10)
        qhigh = min_value + mm_range * (j / 10 + 0.2)
        
        print("j:", j, "qlow:", qlow, "qhigh:", qhigh)
        
                
        if ((df.loc[(df[df.columns[i]] >= qlow) & (df[df.columns[i]] <= qhigh)][df.columns[i]].count()) / count_total) > 0.6:
            flag = True
            column_name = df.columns[i]
            df_drop = df_drop.drop(columns=[column_name]) #We get name of columns from source DataFrame 'df'. And drop columns by this name in 'df_drop'
            break
    print(flag)
    flag = False
    


In [ ]:
df_drop.head(10)

In [ ]:
plt.figure(figsize=(15,8))
sns.pairplot(df_drop, hue="Bankrupt")

# To plot needs time in Kaggle. Be patience. 

In [ ]:
# Train Random Forest or something close to them 
from sklearn import tree

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.metrics import confusion_matrix

from IPython.display import SVG
from graphviz import Source
from IPython.display import display
from IPython.display import Image
#import pydotplus

from IPython.display import HTML
style = "<style>svg{width:70% !important;height:70% !important;}</style>"
HTML(style)


# X and y from df_drop

Use DataFrame df_drop.

It splits to X (source data) and y (result Series). 

In [ ]:
X = df_drop.drop(['Bankrupt'], axis = 1)
y = df_drop['Bankrupt']

In [ ]:
# Dataset Split.
# 1st variant. Split with "train_test_split"
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0)

In [ ]:
# Learn RandomForestClassifier using GridSearchCV
clf_rf = RandomForestClassifier(criterion='entropy', random_state=0)
parametrs = {'n_estimators': range(10, 30, 10),
             'max_depth': range(1, 12, 2),
             'min_samples_leaf': range(1, 7),
             'min_samples_split': range(2, 11, 2)}
grid_search_cv_clf = GridSearchCV(clf_rf, parametrs, cv=3, n_jobs=-1)
grid_search_cv_clf.fit(X_train, y_train)
grid_search_cv_clf.best_params_

{'max_depth': 9,
 'min_samples_leaf': 2,
 'min_samples_split': 8,
 'n_estimators': 10}

In [ ]:
best_clf = grid_search_cv_clf.best_estimator_

In [ ]:
feature_importances_df = pd.DataFrame({'features':list(X_train.columns), 
                                       'feature_importances':  best_clf.feature_importances_})\
                        .sort_values(by='feature_importances')

feature_importances_df.plot.pie(
                        explode=[0.1]*len(X_train.columns),
                        labels = feature_importances_df.features,
                        y = 'feature_importances',
                        autopct='%1.1f%%',
                        shadow=True,
                        legend=False,
                        figsize=(10, 10));

In [ ]:
feature_importances_df.tail()

# features                          feature_importances

Current_Assets/Total_Assets	                0.098624

Total_Asset_Growth_Rate	                    0.111920

CFO_to_Assets	                            0.116886

Inventory_and_accounts_receivable/Net_value	0.118806

Working_Capital_to_Total_Assets	            0.267565


In [ ]:
y_pred = grid_search_cv_clf.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap="Blues")

In [ ]:
# Build Decission Tree
# Use parameters found with RandomForest adn GridSearchCV
# {'max_depth': 9, 'min_samples_leaf': 2, 'min_samples_split': 8


clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=9, min_samples_split=8, min_samples_leaf=2)
clf.fit(X, y)

In [ ]:
graph = Source(tree.export_graphviz(clf, out_file=None, feature_names=list(X), class_names=['Non-Bankrupt', 'Bankrupt'], filled=True))
#graph_png = pydotplus.graph_from_dot_data(graph)
#Image(graph.create_png())
display(SVG(graph.pipe(format='svg')))

In [ ]:
best_clf.score(X_test, y_test)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
precision_score(y_test, y_pred, average=None)

In [ ]:
precision_score(y_test, y_pred, average='binary')

In [ ]:
recall_score(y_test, y_pred, average=None)

In [ ]:
recall_score(y_test, y_pred, average='binary')

In [ ]:
f1_score(y_test, y_pred, average=None)

In [ ]:
# Build a new more compact Decission Tree with "min_samples_leaf = 4" and max_depth=5


clf_compact = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=4, min_samples_leaf=2)
clf_compact.fit(X, y)

In [ ]:
graph = Source(tree.export_graphviz(clf_compact, out_file=None, feature_names=list(X), class_names=['Non-Bankrupt', 'Bankrupt'], filled=True))
#graph_png = pydotplus.graph_from_dot_data(graph)
#Image(graph.create_png())
display(SVG(graph.pipe(format='svg')))